In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time

In [2]:
# 設定 Chrome 瀏覽器選項
def configure_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 如果不需要顯示瀏覽器，可以啟用這行
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument('User-Agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36"')
    return chrome_options

# 初始化 WebDriver
def initialize_driver():
    chrome_options = configure_chrome_options()
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# 訪問網頁並抓取評論資料
def fetch_comments_and_ratings(driver, url):
    driver.get(url)
    
    # 點擊商品評價按鈕
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'goodsCommendLi')]//span"))
    )
    driver.execute_script("arguments[0].click();", review_button)  # 使用 JavaScript 點擊

    time.sleep(3)
    # 獲取總頁數
    page_number_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.pageArea span:nth-child(2)'))
    )
    # 使用正則表達式提取總頁數
    total_pages_match = re.search(r'(\d+)/(\d+)', page_number_element.text)
    
    total_pages = int(total_pages_match.group(2))
    
    # 初始化評論和評分資料
    comments_data = []
    ratings_data = []

    # 抓取評論和評分
    def grab_comments_and_ratings_from_page():
        # 等待評論加載
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.reviewCard'))
        )

        # 抓取評論
        comments = driver.find_elements(By.CSS_SELECTOR, 'div.reviewCardInner div.CommentContainer:not(.ReplyContainer .CommentContainer) p.Comment')
        for comment in comments:
            comments_data.append(comment.text)

        # 抓取評分
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.InfoContainer'))
        )
        rating_star_group = driver.find_elements(By.CSS_SELECTOR, 'div.RatingStarGroup')
        for star in rating_star_group:
            score = star.get_attribute('score')
            ratings_data.append(score)

    # 抓取第一頁評論
    grab_comments_and_ratings_from_page()

    # 抓取剩餘頁面評論
    for page in range(2, total_pages + 1):
        # **修正 XPath 來選擇分頁按鈕**
        next_page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//dd[@pageidx='{page}']/a"))
        )
    
        # **確保按鈕可以被點擊**
        driver.execute_script("arguments[0].click();", next_page_button)


        # 等待評論加載並抓取評論與評分
        grab_comments_and_ratings_from_page()

    # 格式化資料
    comments_data = [[comment] for comment in comments_data]
    ratings_data = [[rating] for rating in ratings_data]

    return comments_data, ratings_data

# 儲存評論和評分到 CSV 文件
def save_to_csv(comments_data, ratings_data, filename):
    # 開啟文件寫入
    with open(filename, mode='w', newline='', encoding='utf-8-sig') as file:
        writer = csv.writer(file)
        writer.writerow(["Comment", "Rating"])  # 寫入標題列

        # 寫入每條評論與相應的評分
        for comment, rating in zip(comments_data, ratings_data):
            writer.writerow([comment[0], rating[0]])  # 假設 comments_data 和 ratings_data 都是嵌套列表

In [7]:
# 主函數執行流程
def main():
    c = 0
    start_time = time.time()
    # 設定要抓取的商品網址
    urls = [
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12694615&Area=search&mdiv=403&oid=1_8&cid=index&kw=%E5%AE%B6%E9%9B%BB',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13554701&mdiv=2169500000-bt_1_016_01-bt_1_016_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13479586&str_category_code=2134500170&mdiv=2134500000-bt_1_017_01-bt_1_017_01_e31&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13200144&str_category_code=2169500082&ctype=B&Area=DgrpCategory&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9932849&str_category_code=2900100157&ctype=B&Area=DgrpCategory&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11869242&mdiv=NEW',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11419137&str_category_code=2163500075&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9875035&str_category_code=2148800007&mdiv=2148800000-bt_1_016_01-bt_1_016_01_e51&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8594150&str_category_code=2148800011&mdiv=2148800000-bt_1_016_01-bt_1_016_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13080565',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13297853',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11452640&str_category_code=2177600022&mdiv=2177600000-bt_1_017_01-bt_1_017_01_e21&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=5993405&Area=search&mdiv=403&oid=1_6&cid=index&kw=%E7%9C%89%E7%AD%86',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10983464&Area=search&mdiv=403&oid=1_11&cid=index&kw=%E7%B2%89%E9%A4%85',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13409435&str_category_code=2134500304&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10873300&Area=search&mdiv=403&oid=1_9&cid=index&kw=simba',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9111715&Area=search&mdiv=403&oid=1_14&cid=index&kw=simba',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10457034&Area=search&mdiv=403&oid=1_22&cid=index&kw=simba',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10646071&str_category_code=2142600043&mdiv=2142600000-bt_1_016_01-bt_1_016_01_e31&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12771029&str_category_code=2173200018&mdiv=2173200000-bt_1_015_01-bt_1_015_01_e41&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9205830&str_category_code=2164500020&mdiv=2164500000-bt_1_017_04-bt_1_017_04_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11087665&str_category_code=2164500053&mdiv=2164500000-bt_1_017_04-bt_1_017_04_e31&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11961320&str_category_code=2164500080&mdiv=2164500000-bt_1_017_04-bt_1_017_04_e21&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12700964&str_category_code=2121200289&mdiv=2121200000-bt_1_013_01-bt_1_013_01_e61&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8077910&str_category_code=2157100051&mdiv=2157100000-bt_1_016_02-bt_1_016_02_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11732990&str_category_code=2132000169&mdiv=2132000000-bt_1_016_01-bt_1_016_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=6820622&Area=search&mdiv=403&oid=1_3&cid=index&kw=%E6%A4%8D%E6%9D%91%E7%A7%80',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9107336&Area=search&mdiv=403&oid=1_10&cid=index&kw=%E6%A4%8D%E6%9D%91%E7%A7%80',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11204082&str_category_code=4400500004&mdiv=2177600000-bt_1_017_01-bt_1_017_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8365866&str_category_code=2177600007&mdiv=2177600000-bt_1_017_01-bt_1_017_01_e31&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11591662&Area=search&mdiv=403&oid=1_13&cid=index&kw=%E7%9C%89%E7%AD%86',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10776780&Area=search&mdiv=403&oid=1_21&cid=index&kw=%E7%9C%89%E7%AD%86',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10540635&Area=search&mdiv=403&oid=1_4&cid=index&kw=%E7%B2%89%E9%A4%85',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8927032&Area=search&mdiv=403&oid=1_17&cid=index&kw=%E7%B2%89%E9%A4%85',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10979366&Area=search&mdiv=403&oid=1_22&cid=index&kw=%E7%B2%89%E9%A4%85',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10496620&Area=search&mdiv=403&oid=1_13&cid=index&kw=%E7%B2%89%E9%A4%85',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11906090&Area=search&mdiv=403&oid=1_16&cid=index&kw=%E9%A0%85%E9%8D%8A',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12593489&Area=search&mdiv=403&oid=1_2&cid=index&kw=%E5%B0%88%E7%A7%91',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11446832&Area=search&mdiv=403&oid=1_1&cid=index&kw=%E5%B0%88%E7%A7%91',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12187111&Area=search&mdiv=403&oid=1_6&cid=index&kw=%E5%B0%88%E7%A7%91',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=5513299&Area=search&mdiv=403&oid=1_1&cid=index&kw=body+shop',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9210560&Area=search&mdiv=403&oid=1_1&cid=index&kw=%E5%80%AA%E9%9D%9C%E6%80%9D',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9260538&Area=search&mdiv=403&oid=1_2&cid=index&kw=%E5%80%AA%E9%9D%9C%E6%80%9D',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8383034&Area=search&mdiv=403&oid=1_16&cid=index&kw=%E9%9B%85%E6%BC%BE',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9258336&Area=search&mdiv=403&oid=1_8&cid=index&kw=%E9%9B%85%E6%BC%BE',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11864162&Area=search&mdiv=403&oid=1_11&cid=index&kw=%E9%9B%85%E6%BC%BE',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10949218&Area=search&mdiv=403&oid=1_4&cid=index&kw=%E6%8D%B7%E5%AE%89%E7%89%B9',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9581913&Area=search&mdiv=403&oid=1_2&cid=index&kw=%E8%99%8E%E7%89%8C',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8043545&Area=search&mdiv=403&oid=1_1&cid=index&kw=%E8%99%8E%E7%89%8C',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12707104&Area=search&mdiv=403&oid=1_10&cid=index&kw=%E8%99%8E%E7%89%8C',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10835271&Area=search&mdiv=403&oid=1_1&cid=index&kw=%E6%9C%AA%E4%BE%86%E5%AF%A6%E9%A9%97%E5%AE%A4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9686323&Area=search&mdiv=403&oid=1_8&cid=index&kw=%E6%9C%AA%E4%BE%86%E5%AF%A6%E9%A9%97%E5%AE%A4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10015673&Area=search&mdiv=403&oid=1_8&cid=index&kw=simba',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9017243&str_category_code=2177600007&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7300184&str_category_code=2118400050&mdiv=2118400000-bt_1_015_01-bt_1_015_01_e31&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=6349347&str_category_code=2118400007&mdiv=2118400000-bt_1_015_01-bt_1_015_01_e41&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9246379&str_category_code=2142600042&mdiv=2142600000-bt_1_016_01-bt_1_016_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=5327725&str_category_code=2178200011&mdiv=2178200000-bt_1_016_01-bt_1_016_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=6089141&str_category_code=2153400009&mdiv=2153400000-bt_1_015_01-bt_1_015_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7588270&str_category_code=2157500010&mdiv=2157500000-bt_1_016_05-bt_1_016_05_e31&ctype=B&sourcePageType=4', 
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7290742&str_category_code=2157300002&mdiv=2157300000-bt_1_017_01-bt_1_017_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9680639&str_category_code=2157300015&mdiv=2157300000-bt_1_017_01-bt_1_017_01_e21&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10851937&str_category_code=4302900399&mdiv=2161600000-bt_1_016_01-bt_1_016_01_e21&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11766187&str_category_code=2173200007&mdiv=2173200000-bt_1_015_01-bt_1_015_01_e71&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=9750941&str_category_code=2900100353&mdiv=2900100000-bt_2_216_03-bt_2_216_03_e13&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12750011&mdiv=NEW',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8385029&str_category_code=2133000009&ctype=B&Area=DgrpCategory&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11986662&str_category_code=2135300041&mdiv=2135300000-bt_1_015_01-bt_1_015_01_e61&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12778302',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10842364',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11284249',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=8530505',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11067766',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7500314',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=6997832',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13018134',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7662805&cid=recitri&oid=BfL&ctype=B&recomd_id=rgc-mbll_normal_1739466149_55245140',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13274447&str_category_code=2148700013&mdiv=2148700000-bt_1_013_01-bt_1_013_01_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12157521&str_category_code=2148700007&mdiv=2148700000-bt_1_013_01-bt_1_013_01_e91&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10537489&str_category_code=3101600520&mdiv=3101600000-bt_2_215_07-bt_2_215_07_e20&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10273225&str_category_code=3101600326&mdiv=3101600000-bt_2_215_08-bt_2_215_08_e6&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=6624013&cid=recitri&oid=BfL&ctype=B&recomd_id=rgc-mbll_normal_1739466149_55245140',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=7254169&cid=recitri&oid=BfL&ctype=B&recomd_id=rgc-mbll_normal_1739466149_55245140',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=5868977&cid=recitri&oid=BfL&ctype=B&recomd_id=rgc-mbll_normal_1739466149_55245140',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11089990&cid=recitri&oid=BfL&ctype=B&recomd_id=rgc-mbll_normal_1739466149_55245140',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12112116&str_category_code=3103100011&ctype=B&Area=DgrpCategory&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=3290593',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10981214&str_category_code=2152900015&mdiv=2152900000-bt_1_016_03-bt_1_016_03_e11&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=11991198&str_category_code=2152900098&mdiv=2152900000-bt_1_016_03-bt_1_016_03_e31&ctype=B&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=12163199&str_category_code=2152900059&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10976461&str_category_code=2152900024&ctype=B&Area=DgrpCategory&sourcePageType=4',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10812203',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=10817950',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13301743',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13430462',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13458937',
        'https://www.momoshop.com.tw/goods/GoodsDetail.jsp?i_code=13132633'
      ]
    
    # 初始化瀏覽器
    driver = initialize_driver()

    # 創建一個 CSV 文件來儲存所有商品的評論
    all_comments_data = []
    all_ratings_data = []

    for url in urls:
        c += 1
        print(c)
        # 抓取每個商品的評論和評分資料
        comments_data, ratings_data = fetch_comments_and_ratings(driver, url)
        
        # 儲存每個商品的評論資料
        all_comments_data.extend(comments_data)
        all_ratings_data.extend(ratings_data)

    # 儲存所有評論和評分到 CSV 文件
    save_to_csv(all_comments_data, all_ratings_data, "comments_and_ratings.csv")

    end_time = time.time()
    execute_time = end_time - start_time
    print(execute_time)
    # 關閉瀏覽器
    driver.quit()

# 執行主函數
if __name__ == "__main__":
    main()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
